In [1]:
def clearall():
    all = [var for var in globals() if var[0] != "_"]
    for var in all:
        del globals()[var]
clearall()

In [2]:
print('Importing Librairies...')
import talib
import numpy as np
import pandas as pd
import pandas_datareader as web
from colorama import Fore, Back, Style
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')
import time
import datetime as dt
import os
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score,roc_curve,confusion_matrix,classification_report
import warnings
warnings.filterwarnings("ignore")
#import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)
print('Librairies imported')
print('')

Importing Librairies...
Using TensorFlow backend.
Librairies imported



In [3]:


___Author___='LumberJack Jyss'
print('Global Optimized LumberJack Environment Motor 55\nLumberJack Jyss 5779(c)')
print(Fore.BLUE,'°0Oo_D.A.G._26_oO0°')
print('BOOST SKAN 55 Version v5.55',Style.RESET_ALL)

#LaDate = input('Date de DL - YYYY-MM-DD ')
LaDate = '2020-02-07'
try:
    os.mkdir('DL_'+LaDate)
except:
    pass

print('')
print('Sraping tickers')
constituents = pd.read_csv('New.csv')
constituents = constituents.sort_values(['Symbol'])
print('Scrap -----> ok')
# PARAMETRES TEMPORELS INITIAUX
start = LaDate[:2]+str(int(LaDate[2:4])-4)+'-'+LaDate[5:]
end = LaDate
print('DATE ORIGINELLE DU DEBUT DE TOUT... :',start)
error = []


try :
    amorceur = pd.read_csv('DL_'+LaDate+'/compteur'+LaDate+'.csv')
    amorceur = amorceur.drop(['Unnamed: 0'],axis=1)
    amorce = constituents[constituents['Symbol']==amorceur.iloc[-1,0]].index[-1]+1
    compteur = pd.read_csv('DL_'+LaDate+'/compteur'+LaDate+'.csv')
except:
    amorce = 0
    compteur = pd.DataFrame(columns = ['Symb.','Name','Sector','Precision'])
       
# SCRAPING DES DONNES BRUTES
def scrap_data(ticker,start,end):
    df = web.DataReader(ticker,'yahoo',start,end)
    df = df.drop(['Close'],axis=1)
    df['Close'] = df['Adj Close']
    df = df.drop(['Adj Close'],axis = 1)
    return(df)

def scrap_vix(start,end):
    df1 = web.DataReader('^VIX','yahoo',start,end)
    df1 = df1.drop(['Close'],axis=1)
    df1['Close'] = df1['Adj Close']
    df1 = df1.drop(['Adj Close'],axis = 1)
    return(df1)

def prepa_data(df,df1):
    rsi = talib.RSI(df['Close'],timeperiod=14)
    stoc_slowk, stoc_slowd = talib.STOCH(df['High'],df['Low'],df['Close'])
    upper, middle, lower =  talib.BBANDS(df['Close'], timeperiod=9, nbdevup=2, nbdevdn=2,matype=0)
    sma5 = talib.SMA(df['Close'],timeperiod=5)
    sma8 = talib.SMA(df['Close'],timeperiod=8)
    sma10 = talib.SMA(df['Close'],timeperiod=10)
    sma12 = talib.SMA(df['Close'],timeperiod=12)
    sma15 = talib.SMA(df['Close'],timeperiod=15)
    sma30 = talib.SMA(df['Close'],timeperiod=30)
    sma35 = talib.SMA(df['Close'],timeperiod=35)
    sma40 = talib.SMA(df['Close'],timeperiod=40)
    sma45 = talib.SMA(df['Close'],timeperiod=45)
    sma50 = talib.SMA(df['Close'],timeperiod=50)
    atr = talib.ATR(df['High'],df['Low'],df['Close'],timeperiod=10)
    delta5_8 = sma5 - sma8
    delta8_10 = sma8 - sma10
    delta10_12 = sma10 - sma12
    delta12_15 = sma12 - sma15
    delta15_30 = sma15 - sma30
    delta30_35 = sma30 - sma35
    delta35_40 = sma35 - sma40
    delta40_45 = sma40 - sma45
    delta45_50 = sma45 - sma50
    bbdelta = upper - middle
    price_bolup = df['Close'] - lower
    price_bolow = df['Close'] - upper
    Ema = talib.EMA(df['Close'],timeperiod=20)
    KC_High = Ema + 2*atr
    KC_Low = Ema - 2*atr
    aroondown, aroonup = talib.AROON(df['High'], df['Low'], timeperiod=9)
    aroon = aroonup - aroondown 
    rsi30_list = []
    rsi70_list = []
    for i in range(0,df.shape[0]):
        rsi70_list.append(70 - rsi[i])
        rsi30_list.append(rsi[i] - 30)
        
    varop_spy = df['Open'] - df['Close'].shift(1)
    varhl_spy = df['High'] - df['Low']
    varop_vix = df1['Open'] - df['Close'].shift(1)
    varhl_vix = df1['High'] - df['Low']
    var_volume = df['Volume'] - df['Volume'].shift(1)

    df['Varop_Spy'] = varop_spy
    df['Varhl_spy'] = varhl_spy
    df['Varop_vix'] = varop_vix
    df['Varhl_vix'] = varhl_vix
    df['Var_Volume'] = var_volume
    df['RSI'] = rsi
    df['70 - RSI'] = np.array(rsi70_list)
    df['RSI - 30'] = np.array(rsi30_list)
    df['BBD_Delta_Up'] = bbdelta
    df['delta5_8'] = delta5_8
    df['delta8_10'] = delta8_10
    df['delta10_12'] = delta10_12
    df['delta12_15'] = delta12_15
    df['delta15_30'] = delta15_30
    df['delta30_35'] = delta30_35
    df['delta35_40'] = delta35_40
    df['delta40_45'] = delta40_45
    df['delta45_50'] = delta45_50
    df['Stoc_Slowk'] = stoc_slowk
    df['Stoc_Slowd'] = stoc_slowd
    df['KC_High'] = KC_High
    df['KC_Low'] = KC_Low
    df['upper'] = upper
    df['lower'] = lower
    df['var_bollup_kchigh'] = upper-KC_High
    df['var_bolllow_kclow'] = lower-KC_Low
    df['Aroon Up'] = aroonup
    df['Aroon Down'] = aroondown
    df['Delta Aroon'] = aroon
    
    trend = []
    
    df = df.dropna()
    df['%Futur'] = ((df['Close'].shift(-1) - df['Close']) *100) / (df['Close'])
    for i in range(0,df.shape[0]):
        if df.iloc[i]['%Futur'] > 0.5 :
            trend.append(1)
        elif df.iloc[i]['%Futur'] < -0.5:
            trend.append(0)
        else:
            trend.append(0)
        
    df['target'] = trend  
    #df = df.dropna()
    
    df = df[['High','Low','Open','Volume','Close','Var_Volume','Varop_Spy','Varhl_spy','Varop_vix','Varhl_vix','RSI',\
             '70 - RSI','RSI - 30','BBD_Delta_Up','delta5_8','delta8_10','delta10_12',\
             'delta12_15','delta15_30','delta30_35','delta35_40','delta40_45','delta45_50',\
             'Stoc_Slowk','Stoc_Slowd','KC_High','KC_Low','upper','lower','var_bollup_kchigh',\
             'var_bolllow_kclow','Aroon Up','Aroon Down','Delta Aroon',\
             '%Futur','target']]
        
    return(df)

def boost(df):
    limit = round(len(df)*.25)
    X = df.copy()
    X = X.drop(['Close'],axis=1)
    X['Close'] = df['Close']
    y = X.iloc[:,-1]
    Xtrain = X.iloc[:-limit,:-1]
    Xtest = X.iloc[-limit:-1,:-1]
    yshift = y.shift(-1)
    ytrain = yshift.iloc[:-limit]
    ytest = yshift.iloc[-limit:-1]

    model = xgb.XGBRegressor(n_estimators=20000, learning_rate=1, gamma=1, subsample=1, colsample_bytree=1, max_depth=100,objective='reg:squarederror')

    model.fit( Xtrain, ytrain, early_stopping_rounds=150, eval_set=[(Xtest, ytest)], verbose=0)

    ytrain_pred = model.predict(Xtrain)

    y_pred = model.predict(Xtest)

    pred = model.predict(X.iloc[:,:-1])

    df['Close.S'] = pred
    df = df.dropna()
    return(df)

    
def deep_learning(df):
    X = df.iloc[:,:-2]
    y = df.iloc[:,-1].values
    X.astype(np.float64)
    scaler = MinMaxScaler(feature_range=(0, 1))
    X = scaler.fit_transform(X)
    y = np.array(y).reshape(-1,1)

    Xtrain = X[:bloc1,:]
    Xtest = X[bloc1:,:]
    ytrain = y[:bloc1,:]
    ytest = y[bloc1:,:]
    
    seed = 770
    np.random.seed(seed)

    ytrain = ytrain.reshape(ytrain.shape[0],)

    Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1])

    model = Sequential()
    # Add an input layer 
    model.add(Dense(1, activation='tanh'))
    # Add one hidden layer 
    model.add(Dense(23, activation='tanh'))
    # Add an input layer 
    model.add(Dense(50, activation='tanh'))
    # Add an input layer 
    model.add(Dense(23, activation='tanh'))
    # Add an output layer 
    model.add(Dense(1, activation='tanh'))

    model.compile(loss= 'categorical_hinge',
                  optimizer='rmsprop', 
                  metrics=['accuracy','mse'])
                  #loss = 'binary_crossentropy'
                  # optimizer = #rmsprop #adam
    
    
    history = model.fit(Xtrain, ytrain,epochs=280, batch_size=8, verbose=0)

    train_acc = model.evaluate(Xtrain, ytrain,verbose=1)

    yhat = model.predict_classes(Xtest)

    score = model.evaluate(Xtest, ytest,verbose=1)

    predict = model.predict(Xtest)

    accuracy = accuracy_score(ytest, yhat)

    # La précision permet de mesurer la capacité du modèle à refuser résultats non-pertinents : vrais_positifs/(vrais_positifs+faux_positifs)
    precision = precision_score(ytest, yhat)  


    # Recall : (vrai_positifs/(vrais_positifs+faux_négatifs))
    recall = recall_score(ytest, yhat) 

    resultats = pd.DataFrame()
    resultats['Date'] = df.index[bloc1:]
    resultats.index= df.index[bloc1:]
    resultats['Move'] = yhat
    resultats['Confiance'] = (predict)*100
    resultats['Actual'] = df.iloc[bloc1:]['Close']
    resultats['Actual.S'] = df.iloc[bloc1:]['Close'].shift(-1)
    open_S = df['Open'].shift(-1)
    resultats['Open.S'] = open_S.iloc[bloc1:]
    
    return(resultats,precision,model,scaler)


def save_model(model):
    savename = 'DL_'+LaDate+'/Save_'+ticker
    # serialize model to YAML
    model_yaml = model.to_yaml()
    with open(savename+".yaml", "w") as yaml_file:
        yaml_file.write(model_yaml)
    # serialize weights to HDF5
    model.save_weights(savename+".h5")
    
########################
#### MAIN SKAN55 #######
########################
ticker_list = compteur['Symb.'].tolist()
name_list = compteur['Name'].tolist()
sector_list = compteur['Sector'].tolist()
prec_list = compteur['Precision'].tolist()

tmps55=time.time()
try:
    print(Fore.BLUE,'Deeping in blue from ',ticker_list[-1],Style.RESET_ALL)
except:
    print(Fore.BLUE,'Deeping in blue from ','A',Style.RESET_ALL)

ERRORPREC = []

#for loop in range(amorce,amorce+25):
for loop in range(amorce,len(constituents)):
    
    try:

        ticker = (constituents.iloc[loop]['Symbol'])
        name = constituents.iloc[loop]['Name']
        sector = constituents.iloc[loop]['Sector']

        global delta,bloc1,bloc2
        tmps1=time.time()
        df = scrap_data(ticker,start,end)

        tmps2=round(time.time()-tmps1,2)
        delta = round(df.shape[0])
        bloc1 = round(delta*0.80)
        bloc2 = delta - bloc1
        #df = boost(df)
        df1 = scrap_vix(start,end)
        df = prepa_data(df,df1)
        aaa = 1
        resultats,precision,model,scaler = deep_learning(df)
        print('\rPassage N°',aaa,'pour le ticker',ticker,'et précision atteinte = ',round(precision*100,2),'%',end='',flush=True)
        
        while (precision*100) < 83 :
            resultats,precision,model,scaler = deep_learning(df)
            print('\rPassage N°',aaa,'pour le ticker',ticker,'et précision atteinte = ',round(precision*100,2),'%',end='',flush=True)
            aaa += 1
            if aaa >= 101:
                ERRORPREC.append((aaa,precision))
                break
                
        if aaa >= 101:
            print('Problème sur le ticker', ticker)
            aaa = 0
            continue
        
        ticker_list.append(ticker)
        name_list.append(name)
        sector_list.append(sector)
        prec_list.append(round(precision*100,2))
        save_model(model)

        print('Le ',Fore.BLUE,'Deep Learning',Style.RESET_ALL ,'de ',Fore.YELLOW,ticker,Style.RESET_ALL,' a été effecué avec succès avec une précision de'\
        ,round((precision * 100),2),'. Les modèles ont été sauvegardés')

    except:
        print(Fore.RED,'Problème loop : ',loop,Style.RESET_ALL)
        error.append((loop,ticker))
         
        continue

print(Fore.YELLOW,Back.BLUE,'Longueur des listes pour vérification : ',len(ticker_list),len(name_list),len(sector_list),Style.RESET_ALL)

compteur = pd.DataFrame(columns = ['Symb.','Name','Sector'])

compteur['Symb.'] = ticker_list

compteur['Name'] = name_list

compteur['Sector'] = sector_list

compteur['Precision'] = prec_list

compteur.to_csv('DL_'+LaDate+'/compteur'+LaDate+'.csv')

print(Fore.YELLOW,Back.MAGENTA,Style.DIM,'PASSAGE FINI!!!!!!',Style.RESET_ALL)
tmps2=round(time.time()-tmps55,2)
print ("Job done in = %f" %tmps2,'seconds')
    

Global Optimized LumberJack Environment Motor 55
LumberJack Jyss 5779(c)
 °0Oo_D.A.G._26_oO0°
BOOST SKAN 55 Version v5.55 

Sraping tickers
Scrap -----> ok
DATE ORIGINELLE DU DEBUT DE TOUT... : 2016-02-07
 Deeping in blue from  AAP 
153/153 [==============================] - 0s 30us/step
Passage N° 100 pour le ticker AAPL et précision atteinte =  0.0 %Problème sur le ticker AAPL
153/153 [==============================] - 0s 25us/step
Passage N° 39 pour le ticker ABBV et précision atteinte =  0.0 % Problème loop :  4 


In [1]:
2+2

4

In [3]:
aaa

NameError: name 'aaa' is not defined